<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#[Tutorial-Title]¶" data-toc-modified-id="[Tutorial-Title]¶-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>[Tutorial Title]¶</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#Initialization" data-toc-modified-id="Initialization-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Initialization</a></span><ul class="toc-item"><li><span><a href="#Ensure-database-is-running" data-toc-modified-id="Ensure-database-is-running-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Ensure database is running</a></span></li><li><span><a href="#Download-and-install-additional-components." data-toc-modified-id="Download-and-install-additional-components.-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Download and install additional components.</a></span></li><li><span><a href="#Connect-to-database." data-toc-modified-id="Connect-to-database.-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Connect to database.</a></span></li><li><span><a href="#Populate-database-with-test-data." data-toc-modified-id="Populate-database-with-test-data.-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Populate database with test data.</a></span></li><li><span><a href="#Create-secondary-indexes-if-necessary" data-toc-modified-id="Create-secondary-indexes-if-necessary-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Create secondary indexes if necessary</a></span></li></ul></li></ul></li><li><span><a href="#[Tutorial-Section]" data-toc-modified-id="[Tutorial-Section]-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>[Tutorial Section]</a></span><ul class="toc-item"><li><span><a href="#[Subsection-1]" data-toc-modified-id="[Subsection-1]-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>[Subsection 1]</a></span></li><li><span><a href="#[Subsection-2]" data-toc-modified-id="[Subsection-2]-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>[Subsection 2]</a></span></li></ul></li><li><span><a href="#[Next-Section]" data-toc-modified-id="[Next-Section]-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>[Next Section]</a></span><ul class="toc-item"><li><span><a href="#[Subsection-1]" data-toc-modified-id="[Subsection-1]-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>[Subsection 1]</a></span></li><li><span><a href="#[Subsection-2]" data-toc-modified-id="[Subsection-2]-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>[Subsection 2]</a></span></li></ul></li><li><span><a href="#Takeaways-and-Conclusion" data-toc-modified-id="Takeaways-and-Conclusion-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Takeaways and Conclusion</a></span></li><li><span><a href="#Clean-up" data-toc-modified-id="Clean-up-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Clean up</a></span></li><li><span><a href="#Further-Exploration-and-Resources" data-toc-modified-id="Further-Exploration-and-Resources-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Further Exploration and Resources</a></span><ul class="toc-item"><li><span><a href="#Next-steps" data-toc-modified-id="Next-steps-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Next steps</a></span></li></ul></li></ul></div>

# Implementing Read-Write Transactions with R-M-W Pattern 
This tutorial explains the need for the Read-Modify-Write Pattern in order to ensure atomicity and isolation of read-write transactions on a single record. 

This notebook requires Aerospike datbase running on localhost and that python and the Aerospike python client have been installed (`pip install aerospike`). Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) for additional details and the docker container.

## Introduction
In Aerospike, the transactional boundaries are single request, single record. While multiple operations may be specified in a single request and on a single record, each such operation is single bin specific and only certain write operations are allowed. Therefore, neither updates involving multiple bins (e.g, a=a+b) nor general logic (e.g., "concatenate alternate letters of a bin and append") are possible as server side operations. Most updates entail R-M-W or Reading the record, Modifying bins on the client side, and then Writing the record updates back to the server. 

The tutorial first shows how the read-write operations can result in lost writes with an exmaple of concurrent updates. 

Then we show how a version check in R-M-W solves the problem by disallowing intereaved writes and thus protecting against lost writes.

## Prerequisites
This tutorial assumes familiarity with the following topics:

[Provide topics and links. For example:]
- [Hello World](hello_world.ipynb)
- [Aerospike Basic Operations](basic_operations.ipynb)

## Initialization

### Ensure database is running
This notebook requires that Aerospike datbase is running. 
[Include the right code cell for Java or Python from the two cells below.] 

In [ ]:
!asd >& /dev/null
!pgrep -x asd >/dev/null && echo "Aerospike database is running!" || echo "**Aerospike database is not running!**"

### Connect to database.

In [ ]:
# import the module
from __future__ import print_function
import aerospike

# Configure the client
config = {
  'hosts': [ ('127.0.0.1', 3000) ],
  'policy' : {'key': aerospike.POLICY_KEY_SEND}
}

# Create a client and connect it to the cluster
try:
  client = aerospike.client(config).connect()
except:
  import sys
  print("failed to connect to the cluster with", config['hosts'])
  sys.exit(1)
print('Client successfully connected to the database.')

### Populate database with test data.
We create a one record with two integer bins "bin-increment" and "bin-set" (the names will become clear below), both initialized to 1.

In [ ]:
namespace = 'test'
tutorial_set = 'rmw-tutorial-set'
bin_inc = 'bin-increment'
bin_set = 'bin-set'
initial_value = 1
# Records are addressable via a tuple of (namespace, set, key)
try:
    for i in range(10):
      # Write the records
      client.put((namespace, tutorial_set, 'id-1'), {bin_inc: initial_value, bin_set: initial_value})
except Exception as e:
  import sys
  print("error: {0}".format(e), file=sys.stderr)

print('Test data populated.')

# The Problem of Lost Writes
In a concurrent setting, multiple clients may be performaing Read-Modify-Write on the same record and get in each other's way. As various R-M-W can interleave, a transaction's write can be overwritten, thus losing that transaction.

We illustrate this effect by running concurrent threads, each thread in a loop:
- Reads the record
- Writes the record by performing a) atomic increment on bin_inc, and b) updating bin_set with the read value of bin_inc + 1. It also reads in the same multi-op request the resulting values of bin_inc and bin_set
- If bin_inc is greater than bin_set, it increments the counter "interleaved_writes" by the difference.

At the end, each thread outputs its interleaved_writes count. In aggregate, these many writes were lost.

In [ ]:
# Code

# Introducing Version Check
The simple R-M-W is modified with how the Write is done: The Write is conditional on the record not having been modified since the Read. It is a "check-and-set (CAS)" like operation that succeeds if the record version is still the same as at the time of Read. Otherwise it fails, and the client must retry the whole R-M-W pattern.

We perform the same concurrent test with the version check at Write. We expect no interleaved_writes reported in any thread.

In [ ]:
# Code

# Takeaways
In the tutorial we showed the need for first, why read-write transactions need to have the R-M-W pattern in Aerospike. We showed with a simple demo, if performed simply, how writes can be lost in a concurrent environment. Then, by introducing the version check logic and syntax, we demostrated how the developer can ensure atomicity and isolation of the read-write transaction.

# Clean up
Remove data and close connection.

In [ ]:
client.truncate(namespace, tutorial_set, 0)
# Close the connection to the Aerospike cluster
client.close()
print('Removed tutorial data. Connection closed.')

# Further Exploration and Resources
For further exploration of transactions support in Aerospike, check out the following resources:

- Blog posts
    - [Developers: Understanding Aerospike Transactions](https://www.aerospike.com/blog/developers-understanding-aerospike-transactions/)
    - [Twelve Do's of Consistency in Aerospike](https://www.aerospike.com/blog/twelve-dos-of-consistency-in-aerospike/)

## Next steps

Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) to run additional Aerospike notebooks. To run a different notebook, download the notebook from the repo to your local machine, and then click on File->Open, and select Upload.